In [1]:
# Modules
import requests
import pandas as pd
import spaceTraders as st
import math

In [2]:
token = "b33e5ca9-b933-43c3-9249-9fe7ea525fc9"
username = "JimHawkins"

In [3]:
user = st.get_user(username)
st.Game.status()

{'status': 'spacetraders is currently online and available to play'}

# Planets

In [4]:
tritus = st.Game.location("OE-PM-TR")
prime = st.Game.location("OE-PM")

# Marketplace

In [24]:
prime_market = pd.DataFrame(prime.marketplace()).sort_values(by='symbol')
tritus_market = pd.DataFrame(tritus.marketplace()).sort_values(by='symbol')

In [28]:
profit_margin = lambda x: (x['sellPricePerUnit_to'] - x['purchasePricePerUnit_from']) / x['volumePerUnit_from']

def how_much_to_buy(unit_volume, hull_capacity):
    return math.trunc(hull_capacity / unit_volume)

def market_compare(from_market, to_market):
    # Do an Inner Join of the Markets on available goods (symbols)
    market_compare = from_market.join(to_market.set_index('symbol'), on="symbol", how="inner", lsuffix="_from", rsuffix="_to")
    # Get the Profit Margins - factoring in volume of the good
    market_compare['profit'] = market_compare.apply(profit_margin, axis=1)
    return market_compare

def best_buy(from_destination, to_destination):
    # Get Markets of each destination as DataFrames
    from_market = pd.DataFrame(from_destination.marketplace())
    to_market = pd.DataFrame(to_destination.marketplace())
    # Get the market comparison
    market_comparison = market_compare(from_market, to_market)
    # Get the record for the best good
    best_good = market_comparison.loc[market_comparison['profit'].idxmax()]
    return {"symbol": best_good['symbol'], "profit": best_good['profit'], "volume": best_good['volumePerUnit_from']}

In [29]:
# Returns the best good to buy, how many units to buy of it and the expected profit
def what_should_I_buy(ship, destination):
    # Get the best good to buy
    symbol = best_buy(st.Game.location(ship.location), st.Game.location(destination))
    # Work out many units to buy
    units_to_buy = how_much_to_buy(symbol['volume'], 80)
    return {"symbol": symbol['symbol'], "units": units_to_buy, "profit": symbol['profit'] * units_to_buy}

In [35]:
what_to_buy = what_should_I_buy(ship, "OE-PM")
# user.new_order(ship.id, what_to_buy['symbol'], what_to_buy['units'])
what_to_buy

# from_market = pd.DataFrame(st.Game.location(ship.location).marketplace())
# to_market = pd.DataFrame(st.Game.location("OE-PM").marketplace())

# market_compare(from_market, to_market)

{'symbol': 'RESEARCH', 'units': 40, 'profit': 20.0}

In [8]:
flight = user.fly(ship.id, "OE-PM")['flightPlan']['id']

In [16]:
user.flight(flight)

{'flightPlan': {'id': 'cknf9m4si11054961bs6efqdikwu',
  'shipId': 'cknegaohp5912821bs6d0ws1xr1',
  'createdAt': '2021-04-13T00:06:57.475Z',
  'arrivesAt': '2021-04-13T00:08:05.456Z',
  'destination': 'OE-PM',
  'departure': 'OE-PM-TR',
  'distance': 4,
  'fuelConsumed': 2,
  'fuelRemaining': 6,
  'terminatedAt': '2021-04-13T00:08:01.005Z',
  'timeRemainingInSeconds': 0}}

In [13]:
cargo=user.ships[1]['cargo']

# Get Good to sell
def get_cargo_to_sell(cargo):
    cargo_not_fuel = lambda x: x['good'] != "FUEL"
    cargo_to_sell=filter(cargo_not_fuel,cargo)
    return cargo_to_sell



In [98]:
# Sell Good
for good in get_cargo_to_sell(cargo):
    print(user.sell_order(ship.id, good['good'], good['quantity']))

{'credits': 138725, 'order': {'good': 'SHIP_PARTS', 'quantity': 20, 'pricePerUnit': 1268, 'total': 25360}, 'ship': {'id': 'cknegaohp5912821bs6d0ws1xr1', 'location': 'OE-PM-TR', 'x': 23, 'y': -28, 'cargo': [{'good': 'FUEL', 'quantity': 8, 'totalVolume': 8}], 'spaceAvailable': 92, 'type': 'GR-MK-I', 'class': 'MK-I', 'maxCargo': 100, 'speed': 1, 'manufacturer': 'Gravager', 'plating': 10, 'weapons': 5}}


In [7]:
shipId = 'cknegaohp5912821bs6d0ws1xr1'

ship = user.get_ship(shipId)